In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
# path='C:/Projects/Hackerearth/Problem 2/'
path=''
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV 
from sklearn import metrics,ensemble,linear_model,neighbors

C:\Users\swapnil.sharma\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\swapnil.sharma\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
data=pd.read_csv(path+'new_trains.csv')
test_data=pd.read_csv(path+'new_test.csv')

In [3]:
data.shape,test_data.shape

((200000, 521), (100000, 520))

In [4]:
data.head()

,segment,no_of_Cities,no_of_dow,no_of_tod,no_of_genre,no_of_title,total_time_spent,dow_1,dow_1_percent,dow_2,...,Jadeja's All-round Show Drives India,Pyar Ka Dard Hai Meetha Meetha Pyara Pyara,Hero,rd TI Kohli Happy with Series WIn,England Have a Warning for Kohli,Dhoni Does It the Mahi Way with,India's Raw Star,LOL! Saha's All Over Smith,Morris's -ball * Deflates RPS,Can't Miss Dhoni Trolls KP
0,0.0,2,7,14,3,60,87386,3412,0.039045,1737,...,0,0,0,0,0,0,0,0,0,0
1,0.0,3,7,16,2,70,16371,5745,0.350925,3346,...,0,0,0,1,0,0,0,0,0,0
2,0.0,1,1,3,2,2,4142,0,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,3,5,6,2,8,8348,658,0.078821,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,2,5,9,3,11,10132,1641,0.161962,480,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data_copy=data.copy()

In [6]:
del data_copy['segment']

In [7]:
target=data['segment']

In [8]:
tainX,testX,train_y,testy=cross_validation.train_test_split(data_copy,target,test_size=.3)
tainX.shape,train_y.shape,testX.shape,testy.shape

((140000, 520), (140000L,), (60000, 520), (60000L,))

In [9]:
train_batch=xgb.DMatrix( tainX, label=train_y)
val_batch=xgb.DMatrix( testX, label=testy)

In [10]:
num_rounds=74

In [11]:
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic','eval_metric':'auc',
        'learning_rate' :0.1, 'n_estimators':140, 'max_depth':6,
 'min_child_weight':1, 'gamma':0, 'subsample':0.8, 'colsample_bytree':0.8,
 'objective': 'binary:logistic', 'nthread':4, 'scale_pos_weight':1,'seed':27}
watchlist  = [ (train_batch,'train'),(val_batch,'eval')]
model = xgb.train(param, train_batch, num_rounds,watchlist,early_stopping_rounds=50) 

[0]	train-auc:0.798693	eval-auc:0.790453
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[1]	train-auc:0.809538	eval-auc:0.797933
[2]	train-auc:0.814405	eval-auc:0.801017
[3]	train-auc:0.816479	eval-auc:0.802526
[4]	train-auc:0.817506	eval-auc:0.802934
[5]	train-auc:0.819012	eval-auc:0.803638
[6]	train-auc:0.820218	eval-auc:0.804325
[7]	train-auc:0.821203	eval-auc:0.804887
[8]	train-auc:0.822139	eval-auc:0.805278
[9]	train-auc:0.823264	eval-auc:0.806035
[10]	train-auc:0.823807	eval-auc:0.805993
[11]	train-auc:0.824612	eval-auc:0.806378
[12]	train-auc:0.825547	eval-auc:0.806835
[13]	train-auc:0.826274	eval-auc:0.807402
[14]	train-auc:0.826854	eval-auc:0.807948
[15]	train-auc:0.827317	eval-auc:0.808028
[16]	train-auc:0.828338	eval-auc:0.80825
[17]	train-auc:0.829231	eval-auc:0.808916
[18]	train-auc:0.829854	eval-auc:0.808863
[19]	train-auc:0.830345	eval-auc:0.809344
[20]	train-auc:0.830968	eval-a

In [12]:
ped2_XGB=model.predict(val_batch)

In [13]:
metrics.roc_auc_score(testy,ped2_XGB)

0.81881150019573901

In [14]:
ada_model=ensemble.AdaBoostRegressor().fit(tainX,train_y)
print 'done'
pred_ada=ada_model.predict(testX)
metrics.roc_auc_score(testy,pred_ada)

done


0.76924615769058069

In [15]:
GBR_model=ensemble.GradientBoostingRegressor().fit(tainX,train_y)
print 'done'
pred_GBR_model=GBR_model.predict(testX)
metrics.roc_auc_score(testy,pred_GBR_model)

done


0.81383586169605826

In [16]:
Line_model=linear_model.LinearRegression().fit(tainX,train_y)
print 'done'
pred_Line_model=Line_model.predict(testX)
metrics.roc_auc_score(testy,pred_Line_model)

done


0.80876811426264272

In [57]:
pred_ada=ada_model.predict(tainX)
pred_GBR_model=GBR_model.predict(tainX)
pred_Line_model=Line_model.predict(tainX)
ped2_XGB=model.predict(train_batch)

new_cola=pd.DataFrame()
new_cola['pred_ada']=pred_ada
new_cola['pred_GBR_model']=pred_GBR_model
new_cola['pred_Line_model']=pred_Line_model
new_cola['ped2_XGB']=ped2_XGB

In [19]:
len(kuch_bh)

60000

In [20]:
kuch_bh=(pred_ada+pred_GBR_model+pred_Line_model+ped2_XGB)/4
metrics.roc_auc_score(testy,kuch_bh)

0.81627217813448449

In [58]:
new_cola.shape,train_y.shape

((140000, 4), (140000L,))

In [59]:
train_batchaaa=xgb.DMatrix( new_cola, label=train_y)
num_rounds=74

In [60]:
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic','eval_metric':'auc',
        'learning_rate' :0.1, 'n_estimators':140, 'max_depth':6,
 'min_child_weight':1, 'gamma':0, 'subsample':0.8, 'colsample_bytree':0.8,
 'objective': 'binary:logistic', 'nthread':4, 'scale_pos_weight':1,'seed':27}

model2 = xgb.train(param, train_batchaaa, num_rounds) 

In [61]:
pred_ada_val=ada_model.predict(testX)
pred_GBR_model_val=GBR_model.predict(testX)
pred_Line_model_val=Line_model.predict(testX)
ped2_XGB_val=model.predict(val_batch)

new_cola_val=pd.DataFrame()
new_cola_val['pred_ada']=pred_ada_val
new_cola_val['pred_GBR_model']=pred_GBR_model_val
new_cola_val['pred_Line_model']=pred_Line_model_val
new_cola_val['ped2_XGB']=ped2_XGB_val

In [70]:
kuch_bh=(pred_ada_val+pred_GBR_model_val+pred_Line_model_val+ped2_XGB_val)/4
metrics.roc_auc_score(testy,kuch_bh)

0.81093642655357945

In [71]:
kuch_bh=(pred_ada_val+pred_GBR_model_val+pred_Line_model_val)/3
metrics.roc_auc_score(testy,kuch_bh)

0.80817360826959783

In [72]:
kuch_bh=(pred_ada_val+pred_GBR_model_val+ped2_XGB_val)/3
metrics.roc_auc_score(testy,kuch_bh)

0.8100865654385635

In [73]:
kuch_bh=(pred_ada_val+pred_Line_model_val+ped2_XGB_val)/3
metrics.roc_auc_score(testy,kuch_bh)

0.81035368049700962

In [74]:
kuch_bh=(pred_GBR_model_val+pred_Line_model_val+ped2_XGB_val)/3
metrics.roc_auc_score(testy,kuch_bh)

0.81151252305874555

In [ ]:
kuch_bh=(pred_ada_val+pred_GBR_model_val+pred_Line_model_val+ped2_XGB_val)/4
metrics.roc_auc_score(testy,kuch_bh)

In [62]:
val_batchaaa=xgb.DMatrix( new_cola_val)

In [63]:
pred333=model2.predict(val_batchaaa)

In [64]:
metrics.roc_auc_score(testy,pred333)

0.79401405468250386

In [ ]:
lala=pd.read_csv('prob2xgb.csv')

In [ ]:
sub=pd.concat([lala['ID'],pd.DataFrame(pred[::,1])],axis=1)
sub.columns=['ID','segment']
sub.to_csv('xgbcv5.csv',index=False)